In [54]:
import konlpy
import nltk
import json
import urllib.request
import requests
import re
from googletrans import Translator

In [218]:
nltk.download('tagsets')
nltk.download('averaged_perceptron_tagger')
from nltk.tag import pos_tag # 영어로 번역된 각 단어들의 품사 구분하기

[nltk_data] Downloading package tagsets to
[nltk_data]     C:\Users\kmk51\AppData\Roaming\nltk_data...
[nltk_data]   Package tagsets is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\kmk51\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [236]:
print("보그체로 변환하고 싶은 문장을 입력하시오")
Korean_Text_b = '이것은 빵입니다. 저것은 음료수입니다. 모두 다 섭취해서 허기진 배를 채우시기를 바랍니다.'
print(Korean_Text_b)

보그체로 변환하고 싶은 문장을 입력하시오
이것은 빵입니다. 저것은 음료수입니다. 모두 다 섭취해서 허기진 배를 채우시기를 바랍니다.


In [237]:
Korean_Text = Korean_Text_b.split(sep=' ')
Korean_Text

['이것은',
 '빵입니다.',
 '저것은',
 '음료수입니다.',
 '모두',
 '다',
 '섭취해서',
 '허기진',
 '배를',
 '채우시기를',
 '바랍니다.']

In [238]:
Toknize_Okt = konlpy.tag.Okt()
Toknize_Hannanum = konlpy.tag.Hannanum()
Toknize_Kkma = konlpy.tag.Kkma()

Toknized_Korean_Okt = []
Toknized_Korean_Hannanum = []
Toknized_Korean_Kkma = []


for i in range(len(Korean_Text)):
    Toknized_Korean_Okt.append(Toknize_Okt.pos(Korean_Text[i]))
    Toknized_Korean_Hannanum.append(Toknize_Hannanum.pos(Korean_Text[i]))
    Toknized_Korean_Kkma.append(Toknize_Kkma.pos(Korean_Text[i]))

    for j in range(len(Toknized_Korean_Okt[i])):
        Toknized_Korean_Okt[i][j] = list(Toknized_Korean_Okt[i][j])

    for j in range(len(Toknized_Korean_Hannanum[i])):
        Toknized_Korean_Hannanum[i][j] = list(Toknized_Korean_Hannanum[i][j])

    for j in range(len(Toknized_Korean_Kkma[i])):
        Toknized_Korean_Kkma[i][j] = list(Toknized_Korean_Kkma[i][j])


In [239]:
Toknized_Korean_Okt

[[['이', 'Determiner'], ['것', 'Noun'], ['은', 'Josa']],
 [['빵', 'Noun'], ['입니다', 'Adjective'], ['.', 'Punctuation']],
 [['저', 'Determiner'], ['것', 'Noun'], ['은', 'Josa']],
 [['음료수', 'Noun'], ['입니다', 'Adjective'], ['.', 'Punctuation']],
 [['모두', 'Noun']],
 [['다', 'Adverb']],
 [['섭취', 'Noun'], ['해서', 'Verb']],
 [['허기진', 'Verb']],
 [['배', 'Noun'], ['를', 'Josa']],
 [['채우시', 'Verb'], ['기를', 'Verb']],
 [['바랍니다', 'Verb'], ['.', 'Punctuation']]]

In [240]:
Toknized_Korean_Hannanum

[[['이것', 'N'], ['은', 'J']],
 [['빵', 'N'], ['이', 'J'], ['ㅂ니다', 'E'], ['.', 'S']],
 [['저것', 'N'], ['은', 'J']],
 [['음료수', 'N'], ['이', 'J'], ['ㅂ니다', 'E'], ['.', 'S']],
 [['모두', 'M']],
 [['다', 'M']],
 [['섭취해서', 'N']],
 [['허기진', 'N']],
 [['배', 'N'], ['를', 'J']],
 [['채우', 'P'], ['시기', 'E'], ['를', 'J']],
 [['바라', 'P'], ['ㅂ니다', 'E'], ['.', 'S']]]

In [241]:
Toknized_Korean_Kkma

[[['이것', 'NP'], ['은', 'JX']],
 [['빵', 'NNG'], ['이', 'VCP'], ['ㅂ니다', 'EFN'], ['.', 'SF']],
 [['저것', 'NP'], ['은', 'JX']],
 [['음료수', 'NNG'], ['이', 'VCP'], ['ㅂ니다', 'EFN'], ['.', 'SF']],
 [['모두', 'MAG']],
 [['다', 'NNG']],
 [['섭취', 'NNG'], ['하', 'XSV'], ['어서', 'ECD']],
 [['허기지', 'VV'], ['ㄴ', 'ETD']],
 [['배', 'NNG'], ['를', 'JKO']],
 [['채우', 'VV'], ['시', 'EPH'], ['기', 'ETN'], ['를', 'JKO']],
 [['바라', 'VV'], ['ㅂ니다', 'EFN'], ['.', 'SF']]]

In [242]:
def ends_with_jongsung(letter): # 받침 유무 확인하기 받침이 있으면 True, 없으면 False 반환
    m = re.search(r"[가-힣|ㄴ]?", letter)
    if m:
        n = m.group()[-1]
        return (ord(n)-ord("가")) % 28 > 0 # 나머지가 있으면 받침 있음. 반환값 생김.
    else:
        return None

In [243]:
ends_with_jongsung("ㄴ")

True

In [244]:
def josa_change(letter, letter2): # 주격 조사 알맞게 change
    josa = ''
    if letter2 == "을" or letter2 == "를":
        josa = "을" if ends_with_jongsung(letter) else "를" # 앞에 받침 있는 경우, 뒤에는 받침이 없는 경우
    elif letter2 == "이" or letter2 == "가":
        josa = "이" if ends_with_jongsung(letter) else "가"
    elif letter2 == "과" or letter2 == "와":
        josa = "과" if ends_with_jongsung(letter) else "와"
    elif letter2 == "은" or letter2 == "는":
        josa = "은" if ends_with_jongsung(letter) else "는"

    return josa

In [245]:
def guanhyeongsa_eomi_change(letter, letter2): # 관형사 어미 알맞게 change
    '''
    if letter2 == "ㅇ"
    guanhyeonsa_eomi = "하는" if ends_with_jongsung(letter) else "는" # 받침 있고, 받침 없고
    guanhyeonsa_eomi = "한"  if ends_with_jongsung(letter) else ""

    return guanhyeonsa_eomi
    '''
    pass

In [254]:
## Kkma 사용하기
'''
valid_grammer_Okt = ['Noun', 'Verb', 'Adjective', 'Determiner', 'Adverb', 'Conjunction', 'Exclamation']
valid_grammer_Hannanum = ['N', 'P', 'M', 'I']
valid_grammer_Kkma = ['NNP', 'NNG', 'NNB', 'NNBC', 'NR', 'NP', 'VV', 'VA', 'VXV', 'VXA', 'VCP', 'VCP', 'VCN', 'MDN', 'MDT', 'MAG', 'MAC', 'IC']
non_valid_grammer_eomi_Kkma = ['EFN', 'EFQ', 'EFO', 'EFA', 'EFI', 'EFR']
'''

valid_grammer_Okt = ['Noun']
valid_grammer_Hannanum = ['N']
valid_grammer_Kkma = ['NNP', 'NNG', 'NNB', 'NNBC', 'NR', 'NP', 'IC']
non_valid_grammer_eomi_Kkma = ['EFN', 'EFQ', 'EFO', 'EFA', 'EFI', 'EFR']


stopwords =['있']


client_id = "DItIw9wyPPPRDPx4Hy46"
client_secret = "L6ZEleX9Mp"
url = "https://openapi.naver.com/v1/papago/n2mt"
url2 = "https://transliterator.herokuapp.com/"
sess = requests.Session()

vogulized_sentence = ''


for word in range(len(Toknized_Korean_Hannanum)):
    for w_divided in range(len(Toknized_Korean_Hannanum[word])):
            if (Toknized_Korean_Hannanum[word][w_divided][1] in valid_grammer_Hannanum) and (len(Toknized_Korean_Hannanum[word][w_divided][0]) > 1):
                print("번역 가능한 품사 단어 : " + Toknized_Korean_Hannanum[word][w_divided][0])
                Toknized_Text_English = urllib.parse.quote(Toknized_Korean_Hannanum[word][w_divided][0])
                data = "source=ko&target=en&text=" + Toknized_Text_English
                request = urllib.request.Request(url)
                request.add_header("X-Naver-Client-Id", client_id)
                request.add_header("X-Naver-Client-Secret", client_secret)
                response = urllib.request.urlopen(request, data=data.encode("utf-8"))
                rescode = response.getcode()

                if rescode==200:
                    response_body = response.read()
                    json_word = json.loads(response_body.decode("utf-8"))
                    result = json_word['message']['result']['translatedText']

                    # 분절된 한국어 단어를 영어로 번역한 후 품사 매칭시키기
                    english = []
                    english.append(result)
                    pos_tag_list = pos_tag(english)

                    # 만약 번역된 영어 단어가 형용사 품사라면
                    if re.match(r'^N', pos_tag_list[0][1]):
                        payload = {'input': pos_tag_list[0][0]}
                        res = sess.post(url2, data=payload)
                        konglish = res.json()['output'].replace('?','') + ''
                        Toknized_Korean_Hannanum[word][w_divided][0] = konglish
                        print(konglish)
                        vogulized_sentence += konglish

                    else:
                        payload = {'input': result}
                        res = sess.post(url2, data=payload)
                        konglish = res.json()['output'].replace('?','')
                        Toknized_Korean_Hannanum[word][w_divided][0] = konglish
                        print(konglish)
                        vogulized_sentence += konglish

                else:
                    print("Error")

            else:
                print("번역 불가능한 품사 단어 : " + Toknized_Korean_Hannanum[word][w_divided][0])
                '''
                if Toknized_Korean_Hannanum[word][w_divided][1] == 'J':
                    Toknized_Korean_Hannanum[word][w_divided][0] = josa_change(Toknized_Korean_Hannanum[word][w_divided-1][0][-1], Toknized_Korean_Hannanum[word][w_divided][0])
                '''
                if Toknized_Korean_Hannanum[word][w_divided][1] == 'E':
                    if Toknized_Korean_Hannanum[word][w_divided][0][0] == "ㅂ" or Toknized_Korean_Hannanum[word][w_divided][0][0] == "ㄴ":
                        Toknized_Korean_Hannanum[word][w_divided][0][0] =
                        Toknized_Korean_Hannanum[word][w_divided][0].replace(Toknized_Korean_Hannanum[word][w_divided][0][0], "", 0)
                        # Toknized_Korean_Hannanum[word][w_divided-1][1][-1] = "냅"
                    elif Toknized_Korean_Hannanum[word][w_divided][0] == "ㄴ다":
                        Toknized_Korean_Hannanum[word][w_divided][0] == "한다"



                vogulized_sentence += str(Toknized_Korean_Hannanum[word][w_divided][0])

    vogulized_sentence += ' '


print(vogulized_sentence)
print(Korean_Text_b)

번역 가능한 품사 단어 : 나이스
나이스
번역 불가능한 품사 단어 : 은
번역 불가능한 품사 단어 : 빵
번역 불가능한 품사 단어 : 이
번역 불가능한 품사 단어 : ㅂ니다


TypeError: 'str' object does not support item assignment

In [247]:
Toknized_Korean_Kkma # 앞에 받침 있으면 하려, 없으면 려

[[['이것', 'NP'], ['은', 'JX']],
 [['빵', 'NNG'], ['이', 'VCP'], ['ㅂ니다', 'EFN'], ['.', 'SF']],
 [['저것', 'NP'], ['은', 'JX']],
 [['음료수', 'NNG'], ['이', 'VCP'], ['ㅂ니다', 'EFN'], ['.', 'SF']],
 [['모두', 'MAG']],
 [['다', 'NNG']],
 [['섭취', 'NNG'], ['하', 'XSV'], ['어서', 'ECD']],
 [['허기지', 'VV'], ['ㄴ', 'ETD']],
 [['배', 'NNG'], ['를', 'JKO']],
 [['채우', 'VV'], ['시', 'EPH'], ['기', 'ETN'], ['를', 'JKO']],
 [['바라', 'VV'], ['ㅂ니다', 'EFN'], ['.', 'SF']]]

In [249]:
translator = Translator(service_urls=[
      'translate.google.co.kr',
    ])
result = translator.translate(Korean_Text_b, src='ko', dest='en')
print(result.text)

This is bread.That is a drink.I hope you all eat and fill your hungry stomach.


In [250]:
Korean_Text_b

'이것은 빵입니다. 저것은 음료수입니다. 모두 다 섭취해서 허기진 배를 채우시기를 바랍니다.'

In [251]:
'''
valid_grammer_Okt = ['Noun', 'Verb', 'Adjective', 'Determiner', 'Adverb', 'Conjunction', 'Exclamation']
valid_grammer_Hannanum = ['N', 'P', 'M', 'I']
valid_grammer_Kkma = ['NNP', 'NNG', 'NNB', 'NNBC', 'NR', 'NP', 'VV', 'VA', 'VXV', 'VXA', 'VCP', 'VCP', 'VCN', 'MDN', 'MDT', 'MAG', 'MAC', 'IC']
non_valid_grammer_eomi_Kkma = ['EFN', 'EFQ', 'EFO', 'EFA', 'EFI', 'EFR']
'''

valid_grammer_Okt = ['Noun']
valid_grammer_Hannanum = ['N']
valid_grammer_Kkma = ['NNP', 'NNG', 'NNB', 'NNBC', 'NR', 'NP', 'IC']
non_valid_grammer_eomi_Kkma = ['EFN', 'EFQ', 'EFO', 'EFA', 'EFI', 'EFR']


stopwords =['있']


translator = Translator(service_urls=['translate.google.co.kr'])
url2 = "https://transliterator.herokuapp.com/"
sess = requests.Session()
vogulized_sentence = ''

for word in range(len(Toknized_Korean_Hannanum)):
    for w_divided in range(len(Toknized_Korean_Hannanum[word])):
            if Toknized_Korean_Hannanum[word][w_divided][1] in valid_grammer_Hannanum and Toknized_Korean_Hannanum[word][w_divided][0] not in stopwords:
                print("번역 가능한 품사 단어 : " + Toknized_Korean_Hannanum[word][w_divided][0])

                konglish = translator.translate(Toknized_Korean_Hannanum[word][w_divided][0], src='ko', dest='en').text
                payload = {'input': konglish}
                res = sess.post(url2, data=payload)
                konglish = res.json()['output'].replace('?','')
                Toknized_Korean_Kkma[word][w_divided][0] = konglish
                print(konglish)

                vogulized_sentence += konglish

            else:
                print("번역 불가능한 품사 단어 : " + Toknized_Korean_Kkma[word][w_divided][0])
                if Toknized_Korean_Kkma[word][w_divided][1] == 'JKS':
                    Toknized_Korean_Kkma[word][w_divided][0] = josa_change(Toknized_Korean_Kkma[word][w_divided-1][0][-1], Toknized_Korean_Kkma[word][w_divided][0])

                if Toknized_Korean_Kkma[word][w_divided][1] == 'ETD' or Toknized_Korean_Kkma[word][w_divided][1] =='EFN':
                    if Toknized_Korean_Kkma[word][w_divided][0] == "ㄴ" :
                        Toknized_Korean_Kkma[word][w_divided][0] == "한"
                    elif Toknized_Korean_Kkma[word][w_divided][0] == "ㄴ다":
                        Toknized_Korean_Kkma[word][w_divided][0] == "한다"



                vogulized_sentence += str(Toknized_Korean_Kkma[word][w_divided][0])

    vogulized_sentence += ' '


print(vogulized_sentence)
print(Korean_Text_b)

# nltk 같은 걸로 영어 품사에 따라 주격조사 은,는,이,가 지정하기
# ex) 맛있는 -> 딜리셔스는 -> 딜리셔스한
# 만약 영어 형용사 + 뒤에 은/는 이면 한으로 주격조사 바꾸기

번역 가능한 품사 단어 : 시스
시시스
번역 불가능한 품사 단어 : 은
번역 가능한 품사 단어 : 빵
브레드
번역 불가능한 품사 단어 : 이
번역 불가능한 품사 단어 : ㅂ니다
번역 불가능한 품사 단어 : .
번역 가능한 품사 단어 : 뎃
데스티네이션
번역 불가능한 품사 단어 : 은
번역 가능한 품사 단어 : 드링크
드링크
번역 불가능한 품사 단어 : 이
번역 불가능한 품사 단어 : ㅂ니다
번역 불가능한 품사 단어 : .
번역 불가능한 품사 단어 : 모두
번역 불가능한 품사 단어 : 다
번역 가능한 품사 단어 : 일에이크잇
일액
번역 가능한 품사 단어 : 헝그리
헝그리
번역 가능한 품사 단어 : 배
십
번역 불가능한 품사 단어 : 를
번역 불가능한 품사 단어 : 채우
번역 불가능한 품사 단어 : 시
번역 불가능한 품사 단어 : 기
번역 불가능한 품사 단어 : 바라
번역 불가능한 품사 단어 : ㅂ니다
번역 불가능한 품사 단어 : .
시시스은 브레드이ㅂ니다. 데스티네이션은 드링크이ㅂ니다. 모두 다 일액 헝그리 십를 채우시기 바라ㅂ니다. 
이것은 빵입니다. 저것은 음료수입니다. 모두 다 섭취해서 허기진 배를 채우시기를 바랍니다.
